In [151]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

np.random.seed(123)

In [152]:
# NETWORK = 'InceptionResNetV2'
# NETWORK = 'InceptionV3'
# NETWORK = 'Resnet'
# NETWORK = 'VGG16'
# NETWORK = 'VGG19'
# NETWORK = 'Xception'
# NETWORK = 'DenseNet201'
# NETWORK = 'EfficientNetB1'
# NETWORK = 'EfficientNetB4'
NETWORK = 'EfficientNetB7'
# NETWORK = 'NASNetLarge'
# NETWORK = 'ConvNeXtBase'

METHOD = "MED-ICA"

In [153]:
WORKING_DIRECTORY = '/datassd/WHOLEIMAGE_MAMIP'
PET_features_filename = f"Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"

features_path = os.path.join(WORKING_DIRECTORY, f"EXTRACTED_FEATURES/ExtractedFeatures_{METHOD}")
processed_features_path = os.path.join(WORKING_DIRECTORY, f"PROCESSED_FEATURES/ProcessedFeatures_{METHOD}")
if not os.path.exists(processed_features_path):
    os.makedirs(processed_features_path)


processed_features_filename = f"Processed_Features_{NETWORK}_MA-MIP_WHOLE-IMAGE.csv"
processed_features_full_filename = os.path.join(processed_features_path, processed_features_filename)

PET_featurs_full_path = os.path.join(features_path, PET_features_filename)

PET_filename = PET_features_filename.split(".")[0]

PET_outcome_data = pd.read_csv(PET_featurs_full_path)

outcome_file = "/datassd/WHOLEIMAGE_MAMIP/hecktor2022_patient_endpoint_training.csv"

In [154]:
PET_outcome_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,-0.001540,-0.012263,-0.005070,-0.001789,0.065678,-0.003893,-0.006939,-0.018583,0.023571,...,-0.017198,0.015989,-0.015848,-0.016805,-0.006490,-0.007654,-0.006778,-0.011092,-0.012213,-0.007182
1,CHUM-002,0.005398,-0.009072,-0.008190,0.021808,0.005742,0.026385,0.016976,-0.013899,-0.001265,...,-0.014159,-0.015305,-0.009247,-0.005843,-0.016437,-0.018223,0.016029,-0.011658,-0.017888,0.003321
2,CHUM-006,-0.001735,-0.010035,-0.007937,-0.010653,0.007058,-0.006109,0.012846,-0.016461,0.058975,...,-0.017727,0.014974,-0.015732,-0.016511,0.009011,-0.009685,0.014486,-0.015933,-0.007761,0.017690
3,CHUM-007,0.027251,-0.011419,-0.004301,0.006742,0.010312,0.011482,0.004292,-0.018652,0.046327,...,-0.020152,0.003005,-0.019046,-0.017577,-0.012888,-0.004911,0.075919,-0.019395,-0.015903,0.008571
4,CHUM-008,-0.008094,-0.018939,-0.011007,0.010538,0.016758,0.008969,0.027616,-0.018542,0.017414,...,-0.018559,0.022261,0.002486,-0.019545,-0.012031,-0.002189,0.006380,-0.017070,-0.014566,-0.004597


# Up to here

Read Outcome csv

In [155]:
OD = pd.read_csv(outcome_file)
OD = OD.iloc[:,:]
OD.head()

,PatientID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [156]:
OD = OD.rename(columns={'PatientID': 'Patient_ID'})
OD.head()

,Patient_ID,Relapse,RFS
0,CHUM-001,0,1704
1,CHUM-002,1,439
2,CHUM-006,0,1186
3,CHUM-007,0,1702
4,CHUM-008,0,1499


In [157]:
OD.columns[0]

'Patient_ID'

In [158]:
first_column = PET_outcome_data.columns[0]
PET_outcome_data = PET_outcome_data.rename(columns={first_column: 'Patient_ID'})

In [159]:
OD.columns[0]


'Patient_ID'

In [160]:
PET_outcome_data.columns[0]

'Patient_ID'

In [161]:
PET_outcome_data.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,CHUM-001,-0.001540,-0.012263,-0.005070,-0.001789,0.065678,-0.003893,-0.006939,-0.018583,0.023571,...,-0.017198,0.015989,-0.015848,-0.016805,-0.006490,-0.007654,-0.006778,-0.011092,-0.012213,-0.007182
1,CHUM-002,0.005398,-0.009072,-0.008190,0.021808,0.005742,0.026385,0.016976,-0.013899,-0.001265,...,-0.014159,-0.015305,-0.009247,-0.005843,-0.016437,-0.018223,0.016029,-0.011658,-0.017888,0.003321
2,CHUM-006,-0.001735,-0.010035,-0.007937,-0.010653,0.007058,-0.006109,0.012846,-0.016461,0.058975,...,-0.017727,0.014974,-0.015732,-0.016511,0.009011,-0.009685,0.014486,-0.015933,-0.007761,0.017690
3,CHUM-007,0.027251,-0.011419,-0.004301,0.006742,0.010312,0.011482,0.004292,-0.018652,0.046327,...,-0.020152,0.003005,-0.019046,-0.017577,-0.012888,-0.004911,0.075919,-0.019395,-0.015903,0.008571
4,CHUM-008,-0.008094,-0.018939,-0.011007,0.010538,0.016758,0.008969,0.027616,-0.018542,0.017414,...,-0.018559,0.022261,0.002486,-0.019545,-0.012031,-0.002189,0.006380,-0.017070,-0.014566,-0.004597


In [162]:
# ranges_list = list()
# for col in PET_outcome_data.iloc[:, 1:]:
#     col_range = PET_outcome_data[col].max() - PET_outcome_data[col].min()
#     ranges_list.append(col_range)
#     print(f"Range of column {col}: {col_range}")
# print(f"maximum range: {max(ranges_list)}")

In [163]:
full_features_df = pd.merge(PET_outcome_data, OD, on=OD.columns[0])

In [164]:
full_features_df.head()

,Patient_ID,0,1,2,3,4,5,6,7,8,...,2552,2553,2554,2555,2556,2557,2558,2559,Relapse,RFS
0,CHUM-001,-0.001540,-0.012263,-0.005070,-0.001789,0.065678,-0.003893,-0.006939,-0.018583,0.023571,...,-0.015848,-0.016805,-0.006490,-0.007654,-0.006778,-0.011092,-0.012213,-0.007182,0,1704
1,CHUM-002,0.005398,-0.009072,-0.008190,0.021808,0.005742,0.026385,0.016976,-0.013899,-0.001265,...,-0.009247,-0.005843,-0.016437,-0.018223,0.016029,-0.011658,-0.017888,0.003321,1,439
2,CHUM-006,-0.001735,-0.010035,-0.007937,-0.010653,0.007058,-0.006109,0.012846,-0.016461,0.058975,...,-0.015732,-0.016511,0.009011,-0.009685,0.014486,-0.015933,-0.007761,0.017690,0,1186
3,CHUM-007,0.027251,-0.011419,-0.004301,0.006742,0.010312,0.011482,0.004292,-0.018652,0.046327,...,-0.019046,-0.017577,-0.012888,-0.004911,0.075919,-0.019395,-0.015903,0.008571,0,1702
4,CHUM-008,-0.008094,-0.018939,-0.011007,0.010538,0.016758,0.008969,0.027616,-0.018542,0.017414,...,0.002486,-0.019545,-0.012031,-0.002189,0.006380,-0.017070,-0.014566,-0.004597,0,1499


In [165]:
full_features_df.to_csv(processed_features_full_filename, encoding='utf-8', index=False)